In [6]:
import warnings
warnings.filterwarnings('ignore')
from gensim.models import KeyedVectors
from urllib import request
import string
from nltk.corpus import brown
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem.porter import *
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from gensim.utils import simple_preprocess
from gensim import corpora
import re
from nltk.stem.porter import PorterStemmer  
from nltk import pos_tag
porter_stemmer = PorterStemmer() 
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')
punc = '[,.!\')(-;?:''"``""]'
from nltk import sent_tokenize, word_tokenize
import pandas as pd
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home_lab_local/s1810410/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import json

def load_dict(file):
    f = open(file, 'r')
    js =f.read()
    DIC = json.loads(js)
    f.close()
    return DIC

# 获取字典信息
f1 = open('corpus/GlossDict.txt', 'r') 
f2 = open('corpus/GlossDictExpand.txt', 'r')
dic = eval(f1.read())   # str into dict
dic2 = eval(f2.read())
glossDict = dic
ExpandglossDict = dic2
f1.close()
f2.close()

# 获取文章信息和文章ID
f3 = open('corpus/ID.txt', 'r')
f4 = open('corpus/INSTANCE.txt', 'r')
id_list = [] # 3944
article = [] # 3944
for line in f3.readlines():
    line = line.strip('\n')
    line = line.strip('"')
    id_list.append(line)

for art in f4.readlines():
    art = art.strip('\n')
    art = art.strip('[,],\\n,",')
    article.append(art)
f3.close()
f4.close()
f6 = open('corpus/GlossDictWITHpunc.txt', 'r') 
dic6 = eval(f6.read())   # str into dict
glossWithPunc = list(dic6)
f6.close()
EPGwithSTOP = open('corpus/EPGwithSTOP.txt', 'r') 
dicEPG = eval(EPGwithSTOP.read())  
EPGwith = list(dicEPG)                                 # wrod 保留stopwords； 额外信息没有stopwords
EPGwithSTOP.close()
sentence_file = open(r"corpus/corp.txt", "r")
out = sentence_file.read()
sentence_corpus = json.loads(out)
sentence_file.close()

In [8]:
glossAND_stop = load_dict('./corpus/glossAND_stop_dict.txt') # 包含stopwords的词典信息
glossDict = dic                                              # 不包含stopwrods的词典信息
hypANDstop = load_dict('./corpus/hypANDstop_dict.txt')    # 加入额外信息的包含stopwords的词典信息
hypAND = load_dict('./corpus/hypAND_dict.txt')            # 加入额外信息的不包含stopwords的词典信息

EPGwith = list(dicEPG)           # 加入的额外信息不包含stopwords

In [9]:
'''new code start '''
def find_depend_collo(target,test):
    # input 一个句子， 一个目标单词
    # 返回 这个句子中 和目标单词有依存关系的 词组
    res = dependency_parser.raw_parse(test)
    tree = list(res.__next__().triples())
    collocations = []
    for child in tree:
        temp = []
        for i in child:
            if len(i)>2:
                temp.append(i)
            else:
                temp.append(i[0])
        if target in temp:
            collocations.append(temp[0]+ ' '+ temp[2]+' '+temp[1])
    return collocations


from collections import Counter
import re
noun_key_value = re.compile(r'[a-z]+[%][\d]+[:][\d]+[:][\d]+[:]+[a-z]*[:]*[\d\d]*')
def RE(S):
    tag = re.findall('\d\d+', S)[0]  # extraction number
    collo = re.sub(' [0-9]+','',S)  #  sub number
    return tag, collo

In [222]:
skip =['he', 'her', 'his', 'a', 'an', 'the', 'that', 'them', 'which', 'her', 'i', 'be', 'my', 'their', 'what', 'where']

In [223]:
# tag answer by dependecy collocation feature
def GetQualtity(jsonfile, LS, i):
    LS = LS[i]
    
    # get quality collo by filt-score
    with open(jsonfile, 'r') as f:
        rawdata = f.readlines()
        rawdata = [i.strip('\n') for i in rawdata]
        rawdata = [i.split(' ') for i in rawdata]
        data = []
        for i in rawdata:
            collo = i[0] + ' ' + i[1] + ' ' + i[3]
            sense = i[4]
            data.append((collo, sense))

    a = {}
    b = {}
    for item in data:
        co = item[0]
        se = item[1]
        if co not in a:
            a[co] = 1
        else:
            a[co] += 1

        co_se = co + ' ' + se
        if co_se not in b:
            b[co_se] = 1
        else:
            b[co_se] += 1

    reliable = {}
    for c,n in b.items():
        k = RE(c)[1]  # RE -> V  ; RE2 -> N
        #k = k.strip()  # N -> remove space
        if k in skip:
            continue
        else:
            if k in a:
                appear_n = n
                score = n/a[k]
                if appear_n > 5 and score >= 0.7:    # (appear_n 5, score 0.7)
                    sense = c.split(' ')[-1]
                    reliable[k] = sense
    
                    
    # tag answer
    res = Find_collo(LS[2], LS[0], LS[1], 9)  # play test data 
    play_ans = []
    
    for context in res[1]:
        flag = 0
        context_co = find_depend_collo(LS[2], ' '.join(context))
        # 存储临时tag， 因为一个句子可能有多个tag，选其中出现次数最多的作为true tag
        temp_tag = {}
        for n,item in enumerate(context_co):
            if item in reliable:  
                tag = reliable[item]  
                if tag not in temp_tag:
                    temp_tag[tag] = 1
                else:
                    temp_tag[tag] += 1              
        # sort reverse select most frequency tag
        if len(temp_tag)>0:
            tag = sorted(temp_tag.items(), key= lambda x:x[1], reverse=True)[0]
            if temp_tag[tag[0]] >= 1:
                play_ans.append(tag)
                flag = 1
        # compute sim by base method if no matching
        if flag == 0:
            tag =  ' ' #Tag_origi(context, glossDict[LS[3]:LS[4]], LS[2], 1)
            play_ans.append(tag)
            
    # write answer into 'test_new'     
    test = play_ans 
    test_new= []   
    for i in test:
        if type(i) == tuple:
            test_new.append(i[0])
        else:
            test_new.append(i)
            
    return test_new, reliable, b, a

In [12]:
# Lemmatizer 词性还原
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(word_tokenize(sentence)):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))
    return res

article = list(map(lambda x: ' '.join(lemmatize_sentence(x)), article))

# clean data with oov, punc, lower characters
def clean(word_list):
    new_sent = word_list
    new_sent = [word for word in new_sent if word in word_vectors.wv.vocab]
    new_sent = [word for word in new_sent if word not in punc]
    new_sent = [w.lower() for w in new_sent]
   #new_sent = [word for word in new_sent if word not in stop_words]
    return new_sent

# delete short sentences in corpus, lemmatize
def txt2wordlist(text):
    afterClean = []
    text = sent_tokenize(text)
    for i in text:
        i = lemmatize_sentence(i)
        i = clean(i)
        if len(i) > 5:
            afterClean.append(i)
    return afterClean

# find collocations for a target word in test data and windowsize words
def Find_collo(keyword,s,e,n):
    collocation = []
    KeyWord_art = []
    target = article[s:e]
    
    # new code
    for art in target:
        art = art.lower()
        seg_sent = sent_tokenize(art)
        for s in seg_sent:
            lem_s = lemmatize_sentence(s)
            lem_s = [w for w in lem_s if w not in punc]
            lem_s = clean(lem_s)
            
            ans = [j for j,x in enumerate(lem_s) if x.find(keyword)!=-1]
            if len(ans)>0:
                ind = ans[0]
                if len(lem_s) > 10 and ind >= 5: 
                    ret = lem_s[ind-4:ind+4]
                elif len(lem_s) > 10 and ind < 5:
                    ret = lem_s[:ind+5]
                else:
                    ret = lem_s
                KeyWord_art.append(ret)
                break
    # new code over
    '''
    for art in target:
        new_sent = lemmatize_sentence(art)
        new_sent = [word for word in new_sent if word not in punc]
        new_sent = [word.lower() for word in new_sent]
        with_stop_sent = new_sent
        
        new_sent = clean(new_sent)
        new_sent = [word for word in new_sent if word not in stop_words] #

        key_index2 = [j for j,x in enumerate(with_stop_sent) if x.find(keyword)!=-1][0]
        key_index =[j for j,x in enumerate(new_sent) if x.find(keyword)!=-1][0]

        key_art = new_sent[(key_index-n):key_index] + new_sent[(key_index):(key_index+n+1)]
        key_collo = with_stop_sent[key_index2-5:key_index2+6] 

        KeyWord_art.append(key_art)
        collocation.append(key_collo)

    '''
    return collocation, KeyWord_art

# extract surrounding texts for a word, n is window size
def surround_context(keyword,corpus,n):
    instance_list = []
    for sent in sentence_corpus:
        key_index =[j for j,x in enumerate(sent) if x == keyword]
        if len(key_index) != 0:
            i = key_index[0]   
            new_list = sent[(i-n):i] + sent[(i):(i+n+1)]
            instance_list.append(new_list)        
    return instance_list

In [13]:
from collections import Counter
import re
noun_key_value = re.compile(r'[a-z]+[%][\d]+[:][\d]+[:][\d]+[:]+[a-z]*[:]*[\d\d]*')
def RE(S):
    tag = re.findall('\d\d+', S)[0]  # extraction number
    collo = re.sub(' [0-9]+','',S)  #  sub number
    return tag, collo

def RE2(S):
    tag = re.search(noun_key_value, S).group()
    collo = re.sub(noun_key_value,'',S)
    return tag, collo

In [14]:
# 为目标单词在corpus中找短语, 
# param: TEXT-> corpus sentecne list, Word-> target word, 
# a,b-> start end index in gloss dict, n-> n pairs for similarity compute
# T -> threshold score of similarity

def tag_collo(TEXT,Word,a,b,n,T):
    score_list = []
    answer_key_list = []
    Score = []
    L_colo = []
    i=0
    while i in range(len(TEXT)):
        sim_score = []
        for d in EPGwith[a:b]:
            sim = SelectWordsSim(d['gloss'], TEXT[i], n) # ith gloss, ith text, nwords
            sim_score.append(sim)
        max_index = sim_score.index(max(sim_score))
        max_score_key = EPGwith[a:b][max_index]['key']  # change dict
        gloss_words = EPGwith[a:b][max_index]['gloss']  # change dict
        max_score = max(sim_score)
        dict={}.fromkeys(('collocation','tag','gloss')) 
        word_index = [j for j,x in enumerate(TEXT[i]) if x.find(Word)!=-1]
        if len(word_index)>0: 
            word_index = word_index[0]
            try:
                COLLO = TEXT[i][word_index-2:word_index+3]
                if max_score > T and len(COLLO)!=0:
                    score_list.append(max_score)
                    answer_key_list.append(str(max_score_key))
                    Score.append(max_score) 
                    dict['collocation'] = COLLO
                    dict['tag'] = max_score_key
                    dict['gloss'] = gloss_words 
                    L_colo.append(dict)
                i = i+1
            except Exception as e:
                i = i+1
        else:
            i = i+1
       
    return L_colo,Score

In [15]:
def Tag_collo_with_filter(word, corpus, n, gs, ge, T, PATTERN):
    SC = surround_context(word, corpus, n)
    SC_LEM = []
    for i in SC:
        SC_txt = ' '.join(i)
        SC_lem = lemmatize_sentence(SC_txt)
        SC_lem = clean(SC_lem)
        SC_LEM.append(SC_lem)
        
    collocations = tag_collo(SC_LEM, word, gs, ge, 8, T)[0]
    c_ = []
    for i in collocations:
        c_tag_d = {}
        a = i['collocation']
        a_tag = i['tag']
        if len(a) == 5:
            try:
                a.index(word)
            except:
                continue
            else:
                c1, c2, c3, c4, c5 = a[:3], a[1:3], a[1:4], a[2:4], a[2:]
                if len(c1)>=2: c_tag_d[' '.join(c5)] = a_tag
                if len(c2)>=2: c_tag_d[' '.join(c4)] = a_tag
                if len(c3)>=2: c_tag_d[' '.join(c3)] = a_tag
                if len(c4)>=2: c_tag_d[' '.join(c2)] = a_tag
                if len(c5)>=2: c_tag_d[' '.join(c1)] = a_tag
            c_.append(c_tag_d)
            
    list1 = []     # list1 is collocations without tag
    list2 = []     # list2 is collocations with tag
    for i in range(len(c_)):
        for k,v in c_[i].items():
            c = k+' '+v
            list1.append(k)
            list2.append(c)
    a=Counter(list1)
    b=Counter(list2)
    left = []
    for k,v in a.items():
        for k1,v1 in b.items():
            k_sub = PATTERN(k1)[1]
            if k == k_sub:
            #if k+' '==k_sub:   # the end of noun collocation with a space
                left.append((k,v,k1,v1))
        
    def takeSecond(elem): # Sort by second element
        return elem[1]  
    left.sort(key=takeSecond, reverse=True)
    
    #LEFT = []
    #for i in left:
    #  if i[1]>=5 and i[3]/i[1] >= 0.6:
    #    LEFT.append(i[2])

    # new rate compute method = p(s|c)/p(s)
    key_list = []
    for ele in left:
        key = PATTERN(ele[2])[0]
        key_list.append(key)
    cont = Counter(key_list) # get all sense and make a dict
    
    LEFT=[]
    scores = []
    for ele in left:
        p_s_c = ele[3]/ele[1]
        key = PATTERN(ele[2])[0]
        sense_count = cont[key]
        p_s = sense_count/sum(cont.values())
        rate = p_s_c/p_s
        
        if rate >= 1 and ele[3]>=4:  # 1.0 and 3 is good (Noun) | 1 and 4 for verb [windows=4, threshold=0.45]
            LEFT.append(ele[2]) 
 
    DICT = {}
    for i,j in map(PATTERN, LEFT):    #v-> RE, n-> RE2
        DICT[j] = i

    verb_delted_list = [word+' what','n\'t '+word, word+' but', 'but '+word, word+' or','or '+word,'the '+word, word+' the', 'in '+word, 'a '+word, word+' a', word+' that', 'that '+word, word+' it', 'it '+word]
    noun_delted_list = ['an '+word+' ', word+' would ', 'be '+word+' ', word+' he ', word+' this ', 'this '+word+' ', word+ ' be ', 'or '+word+' ', word+ ' i ', 'any '+word+' ', 'the '+word+' the ', 'it '+word+' ',word+ ' his ',word+' or ', word+' but ',word+' that ', 'the '+word+' ', word+' the ', 'in '+word+' ', word+' in ', 'at '+'word'+' ', 'for '+word+' ', word+' at ', word+' for ', 'it '+word+' ', word+' it ', 'a '+word+' ', word+' a ', word+' or '] 
    
    for i in noun_delted_list:
        try:
            del DICT[i]
        except Exception as e:
            continue
    return DICT

In [16]:
## weighted sim
from numpy import *
from gensim import matutils
from nltk.parse.stanford import StanfordDependencyParser
myp = '/home_lab_local/s1810410/WSD/stand/stanford-parser-4.0.0/stanford-parser-4.0.0-models.jar'
myp2 = '/home_lab_local/s1810410/WSD/stand/stanford-parser-4.0.0/stanford-parser.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=myp2, path_to_models_jar=myp)

def find_depend_words(target,sent):
    # input 一个句子， 一个目标单词
    # 返回 这个句子中 和目标单词有依存关系的单词列表
    res = dependency_parser.raw_parse(sent)
    tree = list(res.__next__().triples())
    depend_words = []
    for child in tree:
        temp = []
        for i in child:
            temp.append(i[0])
        if target in temp:
            for word in temp:
                if word != target and word in word_vectors.wv.vocab:
                    depend_words.append(word)
    return depend_words
 
def weighted_sim(sent1, sent2, weight_words): 
    # 输入两个句子， 权重单词列表
    # 返回加权之后的 余弦相似度
    sent1 = [word for word in sent1 if word in word_vectors.wv.vocab]
    sent2 = [word for word in sent2 if word in word_vectors.wv.vocab]
    
    v1 = [word_vectors[i]*1.5 if i in weight_words else word_vectors[i] for i in sent1] #如果单词在权重单词中*1.5， 否则不加权重
    v2 = [word_vectors[i] for i in sent2]
    return dot(matutils.unitvec(array(v1).mean(axis=0)), matutils.unitvec(array(v2).mean(axis=0)))

In [17]:
####---over---

In [18]:
# 计算相似度（topn pairs方法）
import heapq
def SelectWordsSim(Glo,Cont,n):
    total = []
    cont_word = []
    for word1 in Glo:
        for word2 in Cont:
            sim = word_vectors.similarity(word1,word2)
            cont_word.append(word2)
            total.append(sim)
    topn = heapq.nlargest(n,total)
    new_words = []
    for i in topn:
        index = total.index(i)
        selected_word = cont_word[index]
        new_words.append(selected_word)
    new_score = word_vectors.n_similarity(Glo,new_words)
    return new_score

# if no collocation has been matched, tag sense-key by computing
# flag =1 原版计算方法, flag=0 加权计算
def Tag_origi(article,gloss, Keyword, flag): 
    scores = []
    for i in range(len(gloss)):
        glo = gloss[i]['gloss']
        if flag==1:
            sim = SelectWordsSim(glo, article, 8) #sim =  word_vectors.n_similarity(glo,article)  # SelectWordsSim(glo, article, 8)
        elif flag==0:
            sent = ' '.join(article)
            weight_words = find_depend_words(Keyword, sent)
            sim = weighted_sim(article, glo, weight_words)
        scores.append(sim)
        index = scores.index(max(scores))
        tag = gloss[index]['key']
    return tag

In [19]:
# 最终整合函数！！！！
def Compute_(Keyword, Collocations, Articles, GS, GE, F):
    target_texts=[]
    Colla=[]
    DataColla=[]
    DATA=()
    DataArt=[]
    Colla = Collocations 
    DataColla = Articles[1]  # 0 collo in test data for target word
    DataArt = Articles[1]    # 1 for orig; 0 for weighted sim 
    DataArt2 = Articles[1]
    
    AN = []   
    colloAN =[]  
    record_id = []
   
    if Colla != None:
        for i,text in enumerate(DataArt):
            flag = False
            t1 = ' '.join(text[3:6])
            t2 = ' '.join(text[4:6])
            t3 = ' '.join(text[4:7])
            t4 = ' '.join(text[5:7])
            t5 = ' '.join(text[5:8])
            t_all = ' '.join(text[3:8])
            for k,v in Colla.items():
                #if t1+' '==k or t2+' '==k or t3+' '==k or t4+' '==k or t5+' '==k:  # noun & adj
                if t1==k or t2==k or t3==k or t4==k or t5==k:                      # verb
                    AN.append(v)
                    colloAN.append(v)
                    record_id.append(i)
                    flag = True
                    break
            if flag == True:
                continue
            else:
                Tag = Tag_origi(DataArt[i], glossDict[GS:GE], Keyword, F)  ## change dict!!  between verb and noun(glossAND_stop)
                AN.append(Tag)
    else:
        for i,text in enumerate(DataArt):
            try:
                Tag = Tag_origi(DataArt[i], glossDict[GS:GE], Keyword, F)  ## change dict!!  between verb and noun
            except Exception as e:
                Tag = Tag_origi(DataArt2[i], glossDict[GS:GE], Keyword, 1)
            AN.append(Tag)
    return AN, colloAN, record_id

In [20]:
def BATCH_compute(keyword, GS, GE, T, AS, AE):
    collocations, articles, answer = None, None, None 
    #collocations = Tag_collo_with_filter(keyword, sentence_corpus, 8, GS,GE,T, RE) 
    articles = Find_collo(keyword, AS, AE, 9)       # 5  for noun and verb  
    answer = Compute_(keyword, collocations, articles, GS, GE, 1)
    return answer[0], answer[1], answer[2]

In [238]:
verb_file = 'corpus/verb.txt'
noun_file = 'corpus/noun.txt'
adj_file = 'corpus/adj.txt'

f_true = pd.read_csv('corpus/v_true.txt',sep=' ',names=['0','1','2','3','4'], header=None, engine='python')
f_noun_true = pd.read_csv('corpus/N_true.txt',sep=' ',names=['0','1','2','3','4'], header=None ,engine='python')
f_adj_true = pd.read_csv('corpus/all_adj_answer.txt',sep=' ',names=['0','1','2','3','4'],header=None,engine='python')
##-
f_answer = pd.read_csv(verb_file, sep=' ', header=None, dtype={0:str})  #修改第0列数据类型为str
f_noun_answer = pd.read_csv(noun_file,sep =' ',header=None)
f_adj_answer = pd.read_csv(adj_file,sep = ' ',header=None)
sense_merge()

VERB_id = []
NOUN_id = []
ADJ_id = []
j = 0
for i in LS_n:           #  LS_n
    id = i[1] - i[0]
    a = [j, id+j]
    NOUN_id.append(a)    # NOUN_id
    j = id+j
    
print('all acc:',newcompare(f_noun_true, f_noun_answer)/1570, '\n', 'success')   # verb:1698 noun:1570

all acc: 0.5445859872611465 
 success


In [239]:
VERB_id = []
j = 0
for i in LS_v:           #  LS_n
    id = i[1] - i[0]
    a = [j, id+j]
    VERB_id.append(a)    # NOUN_id
    j = id+j
    
print('all acc:',newcompare(f_true, f_answer)/1698, '\n', 'success')   # verb:1698 noun:1570

all acc: 0.5871613663133097 
 success


In [127]:
ADJ_id = []
j = 0
for i in LS_adj:           #  LS_n
    id = i[1] - i[0]
    a = [j, id+j]
    ADJ_id.append(a)    # NOUN_id
    j = id+j
    
print('all acc:',newcompare(f_adj_true, f_adj_answer)/141, '\n', 'success')   # verb:1698 noun:1570

all acc: 0.5106382978723404 
 success


In [258]:
import os
a = !ls cd ./dependency/new*
file_list = a[1:]

In [240]:
file_list[0]

'./dependency/new_dependency_activate.txt'

In [ ]:
# verb : 101742 -> 297
# noun : 25151 -> 187
# adj: 5407 -> 44

In [259]:
# 查看 过滤后数量
n = 0
for k, file in enumerate(file_list):
    tem = filter_count(path, LS_v, k)
    n += tem
print(n)
        

297


In [231]:
# 更换答案
import time
start = time.clock()
new_collo = []
temp = []
for k,file in enumerate(file_list):
    path = file
    #s, e = NOUN_id[k][0], NOUN_id[k][1]  #
    s, e = VERB_id[k][0], VERB_id[k][1]  #
    #s, e = ADJ_id[k][0], ADJ_id[k][1]  #
    
    test_new = GetQualtity(path, LS_v, k) # CHANGE 
    play = pd.DataFrame(test_new[0])
    temp.append(test_new[0])    
    print(file + ' successed!') 
    
sense_merge()
end = time.clock()
print('Exe time---:', (end-start)/60,'min')
#print(NUM)

./dependency/new_dependency_activate.txt successed!
./dependency/new_dependency_add.txt successed!
./dependency/new_dependency_appear.txt successed!
./dependency/new_dependency_ask.txt successed!
./dependency/new_dependency_begin.txt successed!
./dependency/new_dependency_climb.txt successed!
./dependency/new_dependency_eat.txt successed!
./dependency/new_dependency_encounter.txt successed!
./dependency/new_dependency_hear.txt successed!
./dependency/new_dependency_lose.txt successed!
./dependency/new_dependency_mean.txt successed!
./dependency/new_dependency_miss.txt successed!
./dependency/new_dependency_play.txt successed!
./dependency/new_dependency_produce.txt successed!
./dependency/new_dependency_provide.txt successed!
./dependency/new_dependency_receive.txt successed!
./dependency/new_dependency_remain.txt successed!
./dependency/new_dependency_rule.txt successed!
./dependency/new_dependency_smell.txt successed!
./dependency/new_dependency_suspend.txt successed!
./dependency/ne

In [188]:
# verb(depd collo 482)

In [232]:
#temp 
new_temp = []
for i in temp:
    for e in i:
        new_temp.append(e)
print(len(new_temp))

id_tag_with_collo = []
num = 0
for i in range(f_answer.shape[0]): # change
    if new_temp[i] != ' ':
        id_tag_with_collo.append(i)
        f_answer[0][i] = new_temp[i]  # change
        num += 1
sense_merge()
print(num) 

1698
482


In [233]:
noun_collo_id = [14, 15, 23, 49, 112, 118, 120, 121, 122, 123, 125, 126, 127, 133, 135, 139, 140, 141, 142, 144, 146, 151, 152, 153, 154, 158, 159, 164, 168, 169, 205, 208, 212, 217, 219, 224, 231, 240, 241, 243, 622, 629, 634, 637, 826, 841, 853, 859, 860, 877, 883, 892, 893, 898, 900, 901, 907, 908, 909, 918, 934, 939, 1443, 1459, 1500, 1505, 1506, 1531]
adj_collo_id = [5, 26, 38, 95, 100, 109]
verb_collo_id = [121, 130, 158, 205, 207, 208, 215, 227, 234, 235, 239, 240, 380, 381, 383, 384, 385, 386, 387, 388, 390, 391, 393, 394, 395, 396, 397, 398, 399, 400, 401, 403, 404, 405, 406, 407, 415, 418, 419, 420, 421, 429, 433, 434, 435, 436, 437, 438, 441, 442, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 482, 483, 484, 485, 488, 489, 490, 491, 492, 494, 495, 498, 500, 503, 504, 505, 506, 507, 508, 509, 661, 664, 669, 670, 672, 683, 687, 689, 691, 693, 694, 695, 697, 698, 701, 704, 705, 708, 712, 717, 723, 725, 728, 734, 811, 813, 815, 823, 825, 833, 836, 837, 839, 841, 843, 844, 849, 851, 860, 863, 872, 957, 971, 977, 982, 993, 1007, 1009, 1011, 1014, 1016, 1020, 1023, 1034, 1039, 1042, 1043, 1044, 1045, 1049, 1050, 1051, 1052, 1061, 1065, 1066, 1069, 1071, 1073, 1075, 1081, 1082, 1089, 1090, 1096, 1098, 1130, 1141, 1155, 1290, 1293, 1295, 1296, 1299, 1315, 1317, 1318, 1322, 1329, 1330, 1331, 1332, 1336, 1539, 1543, 1549, 1636, 1639, 1641, 1642, 1646, 1647, 1648, 1649, 1651, 1652, 1653, 1656, 1661, 1663, 1664, 1666, 1668, 1671, 1672, 1673]

print(len(verb_collo_id))

221


In [234]:
# verb all collo id
all_collo = id_tag_with_collo + verb_collo_id
all_collo = set(all_collo)

In [235]:
co_num = 0
for i in all_collo:
    tans = f_true.iloc[i].values.tolist()[2:]
    ans = f_answer.iloc[i].values
    if ans[0] in tans:
        co_num += 1

print(len(all_collo), co_num / len(all_collo))

618 0.5728155339805825


In [236]:
# 显示单词正确率，整体正确率
def temo_comp(ID, true, answer, pos):
    ret =  {}
    all_counts = []
    all_data = []
    for k,i in enumerate(ID):
        word = pos[k][2]
        counts = 0
        l, r = i[0], i[1]
        subt = true.iloc[l:r]
        suba = answer.iloc[l:r]
        total = subt.shape[0]
        for n in range(total):
            if suba.iloc[n][0] in subt.iloc[n][2:5].values.tolist():
                counts+=1
        acc = round(counts/total , 4)
        all_counts.append(counts)
        all_data.append(total)
        ret[word] = acc
    all_acc = round(sum(all_counts)/sum(all_data), 4)
    fin = sorted(ret.items(), key=lambda x:x[1], reverse=True)
    return fin, all_acc
result = temo_comp(VERB_id, f_true, f_answer, LS_v)
print('--------------')
print(result[1])

--------------
0.5942


In [178]:
v, n , a = 1698, 1570, 141

In [407]:
VERB_id = []
NOUN_id = []
ADJ_id = []
j = 0
for i in LS_v:    # LS_n
    id = i[1] - i[0]
    a = [j, id+j]
    VERB_id.append(a)# NOUN_id
    j = id+j

print('all acc:',newcompare(f_noun_true, f_noun_answer)/15)  

all acc: 0.5471142520612485


In [237]:
print('all acc:',newcompare(f_true, f_answer)/1698)  

all acc: 0.5942285041224971


In [5]:
#parser acc verb 0.561
# parse acc noun 0.541
# parser acc verb 0.541

def AccSort(ID, TRUE, ANSWER, POS):
    #计算单个单词的精确度，观察固定搭配的熵是否有影响
    ret = []
    _,Colcorrect,Collen = comp_single(ID, TRUE, ANSWER)
    for k,i in enumerate(ID):
        word = POS[k][2]
        l,r = i[0], i[1]
        length = r-l
        res = newcompare(TRUE[l:r], ANSWER[l:r])/length         #1570  # accuracy for all noun instances

        collo_acc = Colcorrect[k] / (Collen[k]+0.001)
        rate = Collen[k] / length
        #ret.append([word, round(res,2), round(collo_acc,2), round(rate, 2)])
        ret.append([word, round(res, 2)])
    return ret


def compute_acc(index, tru, ans):   
    # 计算每个单词的collo的acc
    LEN_collo = []
    correct_number = []
    for i in range(len(id_tag_with_collo)):
        INDEX = id_tag_with_collo[i]
        A = index[i] ##
        c = colloComapre(tru, ans, A[0], A[1], INDEX)  ##
        LEN_collo.append(len(INDEX))
        correct_number.append(c)
    if sum(LEN_collo) != 0:
        return sum(LEN_collo), sum(correct_number)/sum(LEN_collo)  # p for tagged by collo insatnces
    else:
        return 0

def newcompare(TRUE,ANSWER):
    correct = []
    for i in range(len(ANSWER)):
        if ANSWER.iloc[i][0] in TRUE.iloc[i][2:5].values:
            correct.append(1)
    return len(correct)

def colloComapre(TRUE, ANSWER, AS, AE, INDEX):
    newTrue = None
    newAnswer = None
    number = None

    newTrue = TRUE[AS:AE]
    newAnswer = ANSWER[AS:AE]
    number = []
    if len(INDEX) == 0:
        return 0
    else:    
        for index in INDEX:
            answer = newAnswer.iloc[index][0]
            Tr = newTrue.iloc[index][2:5].values
            if answer in Tr:
                number.append(1)
    return len(number)


def comp_single(ID, f_tr, f_an):
    LEN_collo = []
    correct_number = []
    
    for i in range(len(id_tag_with_collo)):
        INDEX = id_tag_with_collo[i]
        A = ID[i] ##
        c = colloComapre(f_tr, f_an, A[0], A[1], INDEX)  ##
        LEN_collo.append(len(INDEX))
        correct_number.append(c)
    
    acc = []
    for idx, i in enumerate(LEN_collo):
        if i != 0:
            ac = correct_number[idx]/i
            acc.append(ac)
        else:
            acc.append(0)
    return acc, correct_number, LEN_collo # CHANGE

In [252]:
def filter_count(jsonfile, LS, i):
    LS = LS[i]
    
    # get quality collo by filt-score
    with open(jsonfile, 'r') as f:
        rawdata = f.readlines()
        rawdata = [i.strip('\n') for i in rawdata]
        rawdata = [i.split(' ') for i in rawdata]
        data = []
        for i in rawdata:
            collo = i[0] + ' ' + i[1] + ' ' + i[3]
            sense = i[4]
            data.append((collo, sense))

    a = {}
    b = {}
    for item in data:
        co = item[0]
        se = item[1]
        if co not in a:
            a[co] = 1
        else:
            a[co] += 1

        co_se = co + ' ' + se
        if co_se not in b:
            b[co_se] = 1
        else:
            b[co_se] += 1

    reliable = {}
    for c,n in b.items():
        k = RE(c)[1]  # RE -> V  ; RE2 -> N
        #k = k.strip()  # N -> remove space
        if k in skip:
            continue
        else:
            if k in a:
                appear_n = n
                score = n/a[k]
                if appear_n > 5 and score >= 0.7:    # (appear_n 5, score 0.7)
                    sense = c.split(' ')[-1]
                    reliable[k] = sense
    return len(reliable)

In [253]:
verb_collo_id = [121, 130, 158, 205, 207, 208, 215, 227, 234, 235, 239, 240, 380, 381, 383, 384, 385, 386, 387, 388, 390, 391, 393, 394, 395, 396, 397, 399, 400, 401, 403, 404, 405, 406, 407, 415, 418, 419, 420, 421, 429, 433, 434, 435, 436, 437, 438, 441, 442, 444, 445, 446, 447, 448, 449, 450, 451, 452, 454, 455, 456, 457, 458, 460, 461, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 476, 477, 478, 479, 480, 482, 483, 484, 485, 488, 489, 490, 491, 492, 495, 498, 500, 503, 504, 505, 506, 507, 508, 509, 669, 670, 672, 683, 687, 689, 691, 693, 694, 695, 697, 698, 701, 704, 705, 712, 717, 723, 725, 728, 843, 844, 849, 851, 860, 863, 957, 971, 977, 982, 993, 1007, 1009, 1011, 1014, 1016, 1020, 1023, 1034, 1039, 1042, 1043, 1044, 1045, 1049, 1050, 1051, 1052, 1061, 1065, 1066, 1069, 1071, 1073, 1075, 1081, 1082, 1089, 1090, 1096, 1098, 1130, 1141, 1155, 1290, 1295, 1296, 1299, 1317, 1329, 1330, 1331, 1332, 1636, 1639, 1641, 1642, 1646, 1647, 1648, 1649, 1651, 1652, 1653, 1656, 1661, 1663, 1664, 1666, 1668, 1671, 1672, 1673]

In [176]:
# Noun -> dict expand with stop word; sim T: 0.7, window size: 5, Top n: 8, rate>=1 occur>=3
# acc: 0.794; 34个; all with acc: 0.52
# sim T: 0.65, acc 0.489; 133个, all with acc 0.518
# sim T: 0.6, acc 0.511; 233 个, all with acc 0.52
# sim T: 0.5, acc 0.498; 291个, all with acc 0.514

#newcompare(f_noun_true, f_noun_answer)/1570
#newcompare(f_true, f_answer)/1698     #, newcompare(f_adj_true, f_adj_answer)/141

In [4]:
LS_v = [(0,114,'activate',0,5),(114,246,'add',5,11),(246,379,'appear',11,14),(623,754,'ask',25,31),
     
        (1067,1146,'begin',51,55),(1146,1213,'climb',55,60),(1690,1777,'eat',89,96),
      
        (1777,1842,'encounter',96,100),
      
        (1975,2007,'hear',107,114),(2268,2304,'lose',162,171),(2304,2344,'mean',171,178),(2344,2374,'miss',178,186)
     
        ,(2919,2971,'play',221,233),(2971,3065,'produce',233,239),(3065,3134,'provide',239,245),
     
        (3134,3161,'receive',245,254),(3161,3231,'remain',254,257),(3231,3261,'rule',257,262),(3377,3432,'smell',273,280),
     
        (3589,3653,'suspend',305,312),(3653,3726,'talk',312,321),(3726,3783,'treat',321,330),(3783,3797,'use',330,335),
     
        (3797,3831,'wash',335,347),(3831,3882,'watch',347,354),(3882,3921,'win',354,361),(3921,3944,'write',361,369)]
      

LS_n = [(379,490,'argument',14,19),(490,623,'arm',19,25),
         
         (935,1067,'bank',41,51),(1275,1403,'degree',64,71),(1403,1517,'difference',71,76),(1567,1590,'difficulty',81,85),
         
         (1590,1690,'disc',85,89),(2050,2124,'image',136,143),(2143,2236,'interest',148,155),(2236,2268,'judgment',155,162),
         (2515,2632,'paper',201,208),(2632,2748,'party',208,213),(2748,2835,'performance',213,218),
         
         (2835,2919,'plan',218,221),(3261,3359,'shelter',262,266),(3461,3557,'sort',294,298),(3557,3589,'source',298,305)]
 

LS_adj = [(1517,1567,'different',76,81),(2007,2050,'hot',114,136),
          (2124,2143,'important',143,148),(3432,3461,'solid',280,294)]

In [3]:
#Noune  合并后平均意思3.58， 17个单词， 1570个例子
def sense_merge():
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('argument%1:10:00::','a',inplace=True)
    f_noun_answer.replace('bank%1:06:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:06:01::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:00::','b',inplace=True)
    f_noun_answer.replace('bank%1:14:01::','b',inplace=True)

    f_noun_answer.replace('bank%1:17:00::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:01::','c',inplace=True)
    f_noun_answer.replace('bank%1:17:02::','c',inplace=True)

    f_noun_answer.replace('bank%1:21:00::','d',inplace=True)
    f_noun_answer.replace('bank%1:21:01::','d',inplace=True)

    f_noun_answer.replace('degree%1:07:00::','e',inplace=True)
    f_noun_answer.replace('degree%1:07:01::','e',inplace=True)
    f_noun_answer.replace('degree%1:26:01::','e',inplace=True)

    f_noun_answer.replace('image%1:06:00::','f',inplace=True)
    f_noun_answer.replace('image%1:06:01::','f',inplace=True)

    f_noun_answer.replace('image%1:07:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:00::','g',inplace=True)
    f_noun_answer.replace('image%1:09:02::','g',inplace=True)

    f_noun_answer.replace('interest%1:04:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:01::','h',inplace=True)
    f_noun_answer.replace('interest%1:07:02::','h',inplace=True)
    f_noun_answer.replace('interest%1:09:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:14:00::','h',inplace=True)
    f_noun_answer.replace('interest%1:21:00::','i',inplace=True)
    f_noun_answer.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_answer.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_answer.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_answer.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_answer.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_answer.replace('paper%1:10:01::','l',inplace=True)
    f_noun_answer.replace('paper%1:10:02::','l',inplace=True)
    f_noun_answer.replace('paper%1:06:00::','m',inplace=True)
    f_noun_answer.replace('paper%1:10:03::','m',inplace=True)
    f_noun_answer.replace('paper%1:14:00::','m',inplace=True)

    f_noun_answer.replace('paper%1:10:00::','n',inplace=True)
    f_noun_answer.replace('paper%1:27:00::','n',inplace=True)

    f_noun_answer.replace('party%1:11:00::','o',inplace=True)
    f_noun_answer.replace('party%1:14:02::','o',inplace=True)
    f_noun_answer.replace('party%1:18:00::','o',inplace=True)

    f_noun_answer.replace('performance%1:04:00::','p',inplace=True)
    f_noun_answer.replace('performance%1:04:01::','p',inplace=True)
    f_noun_answer.replace('performance%1:10:00::','p',inplace=True)

    f_noun_answer.replace('performance%1:04:03::','q',inplace=True)
    f_noun_answer.replace('performance%1:22:00::','q',inplace=True)

    f_noun_answer.replace('plan%1:09:00::','r',inplace=True)
    f_noun_answer.replace('plan%1:09:01::','r',inplace=True)

    f_noun_answer.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_answer.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_answer.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_answer.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_answer.replace('source%1:10:00::','u',inplace=True)
    f_noun_answer.replace('source%1:10:01::','u',inplace=True)
    f_noun_answer.replace('source%1:18:00::','u',inplace=True)


    #
    #answer
    #=-----------------
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)
    f_noun_true.replace('argument%1:10:00::','a',inplace=True)

    f_noun_true.replace('bank%1:06:00::','b',inplace=True)
    f_noun_true.replace('bank%1:06:01::','b',inplace=True)
    f_noun_true.replace('bank%1:14:00::','b',inplace=True)
    f_noun_true.replace('bank%1:14:01::','b',inplace=True)

    f_noun_true.replace('bank%1:17:00::','c',inplace=True)
    f_noun_true.replace('bank%1:17:01::','c',inplace=True)
    f_noun_true.replace('bank%1:17:02::','c',inplace=True)

    f_noun_true.replace('bank%1:21:00::','d',inplace=True)
    f_noun_true.replace('bank%1:21:01::','d',inplace=True)

    f_noun_true.replace('degree%1:07:00::','e',inplace=True)
    f_noun_true.replace('degree%1:07:01::','e',inplace=True)
    f_noun_true.replace('degree%1:26:01::','e',inplace=True)

    f_noun_true.replace('image%1:06:00::','f',inplace=True)
    f_noun_true.replace('image%1:06:01::','f',inplace=True)

    f_noun_true.replace('image%1:07:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:00::','g',inplace=True)
    f_noun_true.replace('image%1:09:02::','g',inplace=True)

    f_noun_true.replace('interest%1:04:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:01::','h',inplace=True)
    f_noun_true.replace('interest%1:07:02::','h',inplace=True)
    f_noun_true.replace('interest%1:09:00::','h',inplace=True)
    f_noun_true.replace('interest%1:14:00::','h',inplace=True)
    f_noun_true.replace('interest%1:21:00::','i',inplace=True)
    f_noun_true.replace('interest%1:21:03::','i',inplace=True)

    #------------------
    f_noun_true.replace('judgment%1:04:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:04:02::','j',inplace=True)
    f_noun_true.replace('judgment%1:10:00::','j',inplace=True)
    f_noun_true.replace('judgment%1:07:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:00::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:01::','k',inplace=True)
    f_noun_true.replace('judgment%1:09:04::','k',inplace=True)
    #----------------

    f_noun_true.replace('paper%1:10:01::','l',inplace=True)
    f_noun_true.replace('paper%1:10:02::','l',inplace=True)
    f_noun_true.replace('paper%1:06:00::','m',inplace=True)
    f_noun_true.replace('paper%1:10:03::','m',inplace=True)
    f_noun_true.replace('paper%1:14:00::','m',inplace=True)

    f_noun_true.replace('paper%1:10:00::','n',inplace=True)
    f_noun_true.replace('paper%1:27:00::','n',inplace=True)

    f_noun_true.replace('party%1:11:00::','o',inplace=True)
    f_noun_true.replace('party%1:14:02::','o',inplace=True)
    f_noun_true.replace('party%1:18:00::','o',inplace=True)

    f_noun_true.replace('performance%1:04:00::','p',inplace=True)
    f_noun_true.replace('performance%1:04:01::','p',inplace=True)
    f_noun_true.replace('performance%1:10:00::','p',inplace=True)

    f_noun_true.replace('performance%1:04:03::','q',inplace=True)
    f_noun_true.replace('performance%1:22:00::','q',inplace=True)

    f_noun_true.replace('plan%1:09:00::','r',inplace=True)
    f_noun_true.replace('plan%1:09:01::','r',inplace=True)

    f_noun_true.replace('shelter%1:06:00::','s',inplace=True)
    f_noun_true.replace('shelter%1:06:01::','s',inplace=True)

    f_noun_true.replace('shelter%1:21:00::','t',inplace=True)
    f_noun_true.replace('shelter%1:26:00::','t',inplace=True)

    f_noun_true.replace('source%1:10:00::','u',inplace=True)
    f_noun_true.replace('source%1:10:01::','u',inplace=True)
    f_noun_true.replace('source%1:18:00::','u',inplace=True)

    #Verb  合并后意思 4.07， 27个单词， 1698 个例子

    f_answer.replace('42602','a',inplace=True)
    f_answer.replace('42605','a',inplace=True)
    f_answer.replace('238101','b',inplace=True)
    f_answer.replace('238105','b',inplace=True)
    f_answer.replace('238102','c',inplace=True)
    f_answer.replace('238104','c',inplace=True)
    f_answer.replace('238106','c',inplace=True)  # delete 'begin'
    f_answer.replace('770001','d',inplace=True)
    f_answer.replace('770002','d',inplace=True)   #delete 'decide'
    f_answer.replace('770005','d',inplace=True)
    f_answer.replace('1297001','e',inplace=True)
    f_answer.replace('1297006','e',inplace=True)
    f_answer.replace('1297002','f',inplace=True)
    f_answer.replace('1297007','f',inplace=True)
    f_answer.replace('1353101','g',inplace=True)  #delete expect,express
    f_answer.replace('1353103','g',inplace=True)
    f_answer.replace('1892101','h',inplace=True)
    f_answer.replace('1892103','h',inplace=True)
    f_answer.replace('1892105','h',inplace=True)

    f_answer.replace('1892102','i',inplace=True)
    f_answer.replace('1892104','i',inplace=True)
    f_answer.replace('1892106','i',inplace=True)
    f_answer.replace('1892107','i',inplace=True)

    f_answer.replace('2439901','j',inplace=True)
    f_answer.replace('2439902','j',inplace=True)
    f_answer.replace('2439904','j',inplace=True)
    f_answer.replace('2439905','j',inplace=True)

    f_answer.replace('2439903','k',inplace=True)
    f_answer.replace('2439908','k',inplace=True)
    f_answer.replace('2439906','k',inplace=True)

    f_answer.replace('2555501','l',inplace=True)
    f_answer.replace('2555502','l',inplace=True)
    f_answer.replace('2555507','l',inplace=True)

    f_answer.replace('2555504','m',inplace=True)
    f_answer.replace('2555505','m',inplace=True)
    f_answer.replace('2555506','m',inplace=True)

    f_answer.replace('2644301','n',inplace=True)
    f_answer.replace('2644307','n',inplace=True)
    f_answer.replace('2644302','n',inplace=True)
    f_answer.replace('2644303','n',inplace=True)   # delete note

    f_answer.replace('3165218','o',inplace=True)
    f_answer.replace('3165217','o',inplace=True)
    f_answer.replace('3165213','o',inplace=True)
    
    f_answer.replace('3165220','p',inplace=True)
    f_answer.replace('3165214','p',inplace=True)

    f_answer.replace('3288301','q',inplace=True)
    f_answer.replace('3288306','q',inplace=True)

    f_answer.replace('3288302','r',inplace=True)
    f_answer.replace('3288305','r',inplace=True)

    f_answer.replace('3313901','s',inplace=True)
    f_answer.replace('3313906','s',inplace=True)
    f_answer.replace('3313902','t',inplace=True)
    f_answer.replace('3313905','t',inplace=True)

    f_answer.replace('3434801','u',inplace=True)
    f_answer.replace('3434806','u',inplace=True)

    f_answer.replace('3434803','v',inplace=True)
    f_answer.replace('3434807','v',inplace=True)

    f_answer.replace('3477801','w',inplace=True)
    f_answer.replace('3477803','w',inplace=True)

    f_answer.replace('3597906','x',inplace=True)
    f_answer.replace('3597907','x',inplace=True)
    f_answer.replace('3597911','x',inplace=True)

    f_answer.replace('3893501','y',inplace=True)
    f_answer.replace('3893505','y',inplace=True)
    f_answer.replace('3893507','y',inplace=True)
    f_answer.replace('3893508','y',inplace=True)

    f_answer.replace('3893502','z',inplace=True)
    f_answer.replace('3893503','z',inplace=True)
    f_answer.replace('3893509','z',inplace=True)

    f_answer.replace('4155302','aa',inplace=True)
    f_answer.replace('4155303','aa',inplace=True)
    f_answer.replace('4155304','aa',inplace=True)
    f_answer.replace('4155307','aa',inplace=True)

    f_answer.replace('4198501','bb',inplace=True)
    f_answer.replace('4198502','bb',inplace=True)
    f_answer.replace('4198503','bb',inplace=True)
    f_answer.replace('4198506','bb',inplace=True)
    f_answer.replace('4198507','bb',inplace=True)  

    f_answer.replace('4380101','cc',inplace=True)
    f_answer.replace('4380102','cc',inplace=True)
    f_answer.replace('4380103','cc',inplace=True)

    f_answer.replace('4380104','dd',inplace=True)
    f_answer.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_answer.replace('4380106','ee',inplace=True)
    f_answer.replace('4380109','ee',inplace=True)

    f_answer.replace('4530702','ff',inplace=True)
    f_answer.replace('4530704','ff',inplace=True)

    f_answer.replace('4636102','gg',inplace=True) 
    f_answer.replace('4636101','gg',inplace=True) 
    f_answer.replace('4636107','gg',inplace=True) 
    f_answer.replace('4636108','gg',inplace=True) 

    f_answer.replace('4636103','hh',inplace=True) 
    f_answer.replace('4636104','hh',inplace=True) 
    f_answer.replace('4636110','hh',inplace=True) 
    f_answer.replace('4636111','hh',inplace=True) 
    f_answer.replace('4636112','hh',inplace=True) 
    #---------------


    f_answer.replace('4640501','ii',inplace=True) 
    f_answer.replace('4640502','ii',inplace=True) 
    f_answer.replace('4640507','ii',inplace=True)

    f_answer.replace('4640503','jj',inplace=True) 
    f_answer.replace('4640508','jj',inplace=True) 
    f_answer.replace('4640509','jj',inplace=True) 
    #------------------
    f_answer.replace('4711401','kk',inplace=True) 
    f_answer.replace('4711403','kk',inplace=True) 
    f_answer.replace('4711405','kk',inplace=True)

    f_answer.replace('4711404','ll',inplace=True) 
    f_answer.replace('4711406','ll',inplace=True) 

    #---------------------------------
    #新添加 
    f_answer.replace('4753404','mm',inplace=True) 
    f_answer.replace('4753406','mm',inplace=True)
    f_answer.replace('4753408','mm',inplace=True)

    f_answer.replace('4753403','nn',inplace=True) 
    f_answer.replace('4753407','nn',inplace=True)
    
    f_answer.replace('4753401','oo',inplace=True)
    f_answer.replace('4753405','oo',inplace=True) 

    #
    # 答案
    #---------------------------------------------------------
    f_true.replace('42602','a',inplace=True)
    f_true.replace('42605','a',inplace=True)
    f_true.replace('238101','b',inplace=True)
    f_true.replace('238105','b',inplace=True)
    f_true.replace('238102','c',inplace=True)
    f_true.replace('238104','c',inplace=True)
    f_true.replace('238106','c',inplace=True)  # delete 'begin'
    f_true.replace('770001','d',inplace=True)
    f_true.replace('770002','d',inplace=True)   #delete 'decide'
    f_true.replace('770005','d',inplace=True)
    f_true.replace('1297001','e',inplace=True)
    f_true.replace('1297006','e',inplace=True)
    f_true.replace('1297002','f',inplace=True)
    f_true.replace('1297007','f',inplace=True)
    f_true.replace('1353101','g',inplace=True)  #delete expect,express
    f_true.replace('1353103','g',inplace=True)
    f_true.replace('1892101','h',inplace=True)
    f_true.replace('1892103','h',inplace=True)
    f_true.replace('1892105','h',inplace=True)
    f_true.replace('1892102','i',inplace=True)
    f_true.replace('1892104','i',inplace=True)
    f_true.replace('1892106','i',inplace=True)
    f_true.replace('1892107','i',inplace=True)
    f_true.replace('2439901','j',inplace=True)
    f_true.replace('2439902','j',inplace=True)
    f_true.replace('2439904','j',inplace=True)
    f_true.replace('2439905','j',inplace=True)
    f_true.replace('2439903','k',inplace=True)
    f_true.replace('2439908','k',inplace=True)
    f_true.replace('2439906','k',inplace=True)
    f_true.replace('2555501','l',inplace=True)
    f_true.replace('2555502','l',inplace=True)
    f_true.replace('2555507','l',inplace=True)
    f_true.replace('2555504','m',inplace=True)
    f_true.replace('2555505','m',inplace=True)
    f_true.replace('2555506','m',inplace=True)
    f_true.replace('2644301','n',inplace=True)
    f_true.replace('2644307','n',inplace=True)
    f_true.replace('2644302','n',inplace=True)
    f_true.replace('2644303','n',inplace=True)   # delete note
    f_true.replace('3165218','o',inplace=True)
    f_true.replace('3165217','o',inplace=True)
    f_true.replace('3165213','o',inplace=True)
    f_true.replace('3165220','p',inplace=True)
    f_true.replace('3165214','p',inplace=True)
    f_true.replace('3288301','q',inplace=True)
    f_true.replace('3288306','q',inplace=True)
    f_true.replace('3288302','r',inplace=True)
    f_true.replace('3288305','r',inplace=True)
    f_true.replace('3313901','s',inplace=True)
    f_true.replace('3313906','s',inplace=True)
    f_true.replace('3313902','t',inplace=True)
    f_true.replace('3313905','t',inplace=True)
    f_true.replace('3434801','u',inplace=True)
    f_true.replace('3434806','u',inplace=True)
    f_true.replace('3434803','v',inplace=True)
    f_true.replace('3434807','v',inplace=True)
    f_true.replace('3477801','w',inplace=True)
    f_true.replace('3477803','w',inplace=True)
    f_true.replace('3597906','x',inplace=True)
    f_true.replace('3597907','x',inplace=True)
    f_true.replace('3597911','x',inplace=True)
    f_true.replace('3893501','y',inplace=True)
    f_true.replace('3893505','y',inplace=True)
    f_true.replace('3893507','y',inplace=True)
    f_true.replace('3893508','y',inplace=True)
    f_true.replace('3893502','z',inplace=True)
    f_true.replace('3893503','z',inplace=True)
    f_true.replace('3893509','z',inplace=True)
    f_true.replace('4155302','aa',inplace=True)
    f_true.replace('4155303','aa',inplace=True)
    f_true.replace('4155304','aa',inplace=True)
    f_true.replace('4155307','aa',inplace=True)
    f_true.replace('4198501','bb',inplace=True)
    f_true.replace('4198502','bb',inplace=True)
    f_true.replace('4198503','bb',inplace=True)
    f_true.replace('4198506','bb',inplace=True)
    f_true.replace('4198507','bb',inplace=True)  
    f_true.replace('4380101','cc',inplace=True)
    f_true.replace('4380102','cc',inplace=True)
    f_true.replace('4380103','cc',inplace=True)
    f_true.replace('4380104','dd',inplace=True)
    f_true.replace('4380105','dd',inplace=True)

    #--------------------------------------------
    f_true.replace('4380106','ee',inplace=True)
    f_true.replace('4380109','ee',inplace=True)
    f_true.replace('4530702','ff',inplace=True)
    f_true.replace('4530704','ff',inplace=True)

    f_true.replace('4636102','gg',inplace=True) 
    f_true.replace('4636101','gg',inplace=True) 
    f_true.replace('4636107','gg',inplace=True) 
    f_true.replace('4636108','gg',inplace=True) 
    f_true.replace('4636103','hh',inplace=True) 
    f_true.replace('4636104','hh',inplace=True) 
    f_true.replace('4636110','hh',inplace=True) 
    f_true.replace('4636111','hh',inplace=True) 
    f_true.replace('4636112','hh',inplace=True) 
    #---------------

    f_true.replace('4640501','ii',inplace=True) 
    f_true.replace('4640502','ii',inplace=True) 
    f_true.replace('4640507','ii',inplace=True)
    f_true.replace('4640503','jj',inplace=True) 
    f_true.replace('4640508','jj',inplace=True) 
    f_true.replace('4640509','jj',inplace=True) 
    #------------------
    f_true.replace('4711401','kk',inplace=True) 
    f_true.replace('4711403','kk',inplace=True) 
    f_true.replace('4711405','kk',inplace=True)

    f_true.replace('4711404','ll',inplace=True) 
    f_true.replace('4711406','ll',inplace=True) 

    #---------------------------------
    f_true.replace('4753401','mm',inplace=True) 
    f_true.replace('4753404','mm',inplace=True) 
    f_true.replace('4753406','mm',inplace=True)
    f_true.replace('4753403','nn',inplace=True) 
    f_true.replace('4753407','nn',inplace=True)
    f_true.replace('4753405','oo',inplace=True) 
    f_true.replace('4753408','oo',inplace=True)


    #ADJ  合并后意思 2.75， 141个例子， 4个单词

    f_adj_answer.replace('hot%3:00:01::','a',inplace=True)
    f_adj_answer.replace('hot%3:00:02::','a',inplace=True)
    f_adj_answer.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_answer.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_answer.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_answer.replace('important%3:00:00::','c',inplace=True)
    f_adj_answer.replace('important%3:00:02::','c',inplace=True)
    f_adj_answer.replace('important%3:00:04::','c',inplace=True)
    f_adj_answer.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_answer.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_answer.replace('solid%3:00:01::','e',inplace=True)
    f_adj_answer.replace('solid%3:00:02::','e',inplace=True)
    f_adj_answer.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_answer.replace('solid%5:00:00:cubic:00','f',inplace=True)

    #
    #answer
    #-----------------

    f_adj_true.replace('hot%3:00:01::','a',inplace=True)
    f_adj_true.replace('hot%3:00:02::','a',inplace=True)
    f_adj_true.replace('hot%5:00:00:active:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:charged:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fast:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:fresh:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:good:01','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:illegal:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:lucky:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:near:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:new:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:popular:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:pungent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:radioactive:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:sexy:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:skilled:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:unpleasant:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:violent:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:wanted:00','b',inplace=True)
    f_adj_true.replace('hot%5:00:00:warm:03','b',inplace=True)
    f_adj_true.replace('hot%5:00:02:fast:01','b',inplace=True)
    f_adj_true.replace('important%3:00:00::','c',inplace=True)
    f_adj_true.replace('important%3:00:02::','c',inplace=True)
    f_adj_true.replace('important%3:00:04::','c',inplace=True)
    f_adj_true.replace('important%5:00:00:immodest:02','d',inplace=True)
    f_adj_true.replace('important%5:00:00:influential:00','d',inplace=True)
    f_adj_true.replace('solid%3:00:01::','e',inplace=True)
    f_adj_true.replace('solid%3:00:02::','e',inplace=True)
    f_adj_true.replace('solid%5:00:00:homogeneous:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:honorable:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:opaque:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:plain:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:sound:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:unbroken:02','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:undiversified:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:wholesome:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:hard:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:good:01','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:frozen:00','f',inplace=True)
    f_adj_true.replace('solid%5:00:00:cubic:00','f',inplace=True)